# Notebook 01 — EDA e Limpeza do Dataset **Distribuição de Renda**

**Objetivo:**  
Explorar o dataset bruto (`data/raw/distribuicao-renda.csv`), realizar a análise exploratória de dados (EDA), tratar problemas de qualidade (nulos, outliers, tipos) e gerar um dataset limpo para uso posterior na clusterização.

**Etapas:**
1. Carregamento e inspeção inicial.  
2. Estatísticas descritivas e distribuição das variáveis.  
3. Identificação e tratamento de nulos.  
4. Identificação e tratamento de outliers.  
5. Salvamento de versão limpa em `data/processed/`.  


In [2]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path

# Configurações globais
pd.set_option("display.max_columns", None)
pd.set_option("display.width", 120)

# Paths
ROOT = Path.cwd().resolve().parents[0] if Path.cwd().name == "notebooks" else Path.cwd()
DATA_RAW = ROOT / "data" / "raw"
DATA_PROCESSED = ROOT / "data" / "processed"
DATA_PROCESSED.mkdir(parents=True, exist_ok=True)

RAW_FILE = DATA_RAW / "distribuicao-renda.csv"
print("Arquivo bruto:", RAW_FILE.exists(), RAW_FILE)


Arquivo bruto: True C:\Users\renan\OneDrive\Desktop\kmeans-3d-cluster\data\raw\distribuicao-renda.csv


## 1. Importação de pacotes e configuração de paths


## 2. Carregamento inicial do dataset bruto
- Verificar número de linhas e colunas.  
- Conferir nomes das variáveis.  
- Observar amostra inicial.


## 3. Checagens iniciais
- Tipos de dados de cada coluna.  
- Presença de valores nulos.  
- Presença de duplicados.  


## 4. Estatísticas descritivas
- Medidas resumo para todas as variáveis numéricas.  
- Distribuições (histogramas).  
- Relações entre variáveis (correlação).  


## 5. Tratamento de valores nulos
- Estratégia adotada (remoção, imputação, preenchimento com média/mediana).  
- Justificativa da decisão.  


## 6. Tratamento de outliers
- Método escolhido (ex.: IQR, Z-score, winsorização).  
- Justificativa da decisão.  


## 7. Dataset limpo
- Visualizar primeiras linhas após tratamento.  
- Confirmar que não existem mais nulos.  
- Confirmar coerência das distribuições.  


## 8. Salvamento do dataset limpo
Salvar em `data/processed/distribuicao-renda-clean.csv` para uso posterior na clusterização.


## 9. Próximos passos
- Avaliar se a clusterização será feita:
  - Com todos os dados disponíveis,  
  - Com dois anos específicos,  
  - Ou com apenas um ano.  

Essa decisão será baseada na análise criteriosa do dataset limpo.
